<div class="alert alert-block alert-info">
<span style="color: rgb(0,53,91);">
<center><img src="https://www.redage.org/sites/default/files/styles/large/public/img-logo-institucion/logo_iteso.jpg?itok=IzloBJS1" style="width:500px;height:142px;" title="Logo ITESO"></center>

<font face = "Times New Roman" size = "6"><b><center>Programacion para Mineria de Datos</center></b></font>
<font face = "Times New Roman" size = "5"><b><center>Preprocesamiento PCA</center></b></font>

<b><font back = "Times New Roman" size = "4"><center>Examen Parcial 3</center></font>
<div align="right"><font face = "Times New Roman" size = "2">nombres: Angel Aceves Manzo</font></div>
<div align="right"><font face = "Times New Roman" size = "2"> Jorge Arturo Ponce Martin</font></div>
<div align="right"><font face = "Times New Roman" size = "2"> Jesus Vargas Pacheco</font></div>
</span></div>

# 01_A_Preprocessing_Production

## Instalar librerías en caso de ser necesario

In [ ]:
%pip install numpy pandas matplotlib seaborn scikit-learn joblib

## Importar librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
import joblib
import json
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


## 1. Cargar Artefactos de Preprocesamiento

In [ ]:
scaler = joblib.load('../preprocessing/preprocessor_cat.joblib')
pca = joblib.load('../preprocessing/pca_artifacts_bundle.zip')

# Cargar metadatos
with open('../preprocessing/expected_columns.json', 'r') as f:
    expected_columns = json.load(f)

with open('../preprocessing/pca_metadata.json', 'r') as f:
    pca_metadata = json.load(f)

# Output de status
print("Artefactos cargados correctamente:")
print(f"  - Scaler: {type(scaler).__name__}")
print(f"  - PCA: {type(pca).__name__}")
print(f"  - Número de componentes PCA: {pca_metadata['n_components']}")
print(f"  - Features esperados: {len(expected_columns['encoded_columns'])}")

Artefactos cargados correctamente:
  - Scaler: StandardScaler
  - PCA: PCA
  - Número de componentes PCA: 14
  - Features esperados: 18


## 2. Cargar Datos Desconocidos

In [ ]:
df_desconocidos = pd.read_csv('../../data/Saratoga_desconocidos.csv')

print(f"Datos desconocidos cargados:")
print(f"  - Dimensiones: {df_desconocidos.shape}")
print(f"\nPrimeras filas:")
df_desconocidos.head()

Datos desconocidos cargados:
  - Dimensiones: (10, 16)

Primeras filas:


,lotSize,age,landValue,livingArea,pctCollege,bedrooms,fireplaces,bathrooms,rooms,heating,fuel,sewer,waterfront,newConstruction,centralAir,price
0,0.28,57,23200,1121,61,3,0,1.5,4,hot air,oil,septic,No,No,No,NaN
1,0.81,0,7300,2288,61,3,1,2.5,8,hot air,gas,public/commercial,No,Yes,No,NaN
2,0.41,5,43400,2718,61,4,1,2.5,12,hot air,gas,public/commercial,No,No,Yes,NaN
3,0.58,42,31400,2464,61,4,1,2.5,8,hot water/steam,gas,public/commercial,No,No,No,NaN
4,0.43,41,45800,2426,61,4,2,2.0,7,hot water/steam,gas,public/commercial,No,No,No,NaN


In [4]:
# Eliminar la columna price (vacía en datos desconocidos)
if 'price' in df_desconocidos.columns:
    df_desconocidos_clean = df_desconocidos.drop('price', axis=1)
else:
    df_desconocidos_clean = df_desconocidos.copy()

print(f"Dimensiones después de eliminar price: {df_desconocidos_clean.shape}")

Dimensiones después de eliminar price: (10, 15)


## 3. Preprocesar Datos

In [5]:
# Identificar variables categóricas y numéricas
variables_categoricas = expected_columns['categorical_columns']
variables_numericas = expected_columns['numerical_columns']
# Remover 'price' de variables_numericas si existe
variables_numericas = [col for col in variables_numericas if col != 'price']

print(f"Variables numéricas: {len(variables_numericas)}")
print(f"Variables categóricas: {len(variables_categoricas)}")

Variables numéricas: 9
Variables categóricas: 6


In [6]:
# Codificar variables categóricas (one-hot encoding)
df_encoded = df_desconocidos_clean.copy()

for col in variables_categoricas:
    if col in df_encoded.columns:
        dummies = pd.get_dummies(df_encoded[col], prefix=col, drop_first=True)
        df_encoded = pd.concat([df_encoded, dummies], axis=1)
        df_encoded.drop(col, axis=1, inplace=True)

print(f"Dimensiones después de codificación: {df_encoded.shape}")

Dimensiones después de codificación: (10, 14)


In [7]:
# Asegurar que todas las columnas esperadas estén presentes
expected_cols = expected_columns['encoded_columns']

# Agregar columnas faltantes con valor 0
for col in expected_cols:
    if col not in df_encoded.columns:
        df_encoded[col] = 0

# Eliminar columnas que no se esperan y reordenar
df_encoded = df_encoded[expected_cols]

print(f"Dimensiones finales después de alinear columnas: {df_encoded.shape}")
print(f"Columnas esperadas: {len(expected_cols)}")
print(f"Columnas presentes: {len(df_encoded.columns)}")
print(f"Match: {list(df_encoded.columns) == expected_cols}")

Dimensiones finales después de alinear columnas: (10, 18)
Columnas esperadas: 18
Columnas presentes: 18
Match: True


In [ ]:
# Escalar los datos usando el scaler entrenado
X_scaled = scaler.transform(df_encoded)
X_scaled_df = pd.DataFrame(X_scaled, columns=df_encoded.columns, index=df_encoded.index)

# Output de confirmación
print("Datos escalados correctamente.")
print(f"Media de datos escalados: {X_scaled.mean():.6f}")
print(f"Desviación estándar de datos escalados: {X_scaled.std():.6f}")

Datos escalados correctamente.
Media de datos escalados: -0.150766
Desviación estándar de datos escalados: 1.052796


## 4. Aplicar PCA

In [ ]:
# Aplicar PCA usando el modelo entrenado
X_pca = pca.transform(X_scaled)

# Crear DataFrame con componentes principales
pca_columns = pca_metadata['pca_features']
X_pca_df = pd.DataFrame(X_pca, columns=pca_columns, index=df_encoded.index)

# Output de confirmación
print(f"PCA aplicado correctamente.")
print(f"Dimensiones después de PCA: {X_pca_df.shape}")
print(f"Componentes principales: {len(pca_columns)}")
print(f"\nPrimeras filas con PCA:")
X_pca_df.head()

PCA aplicado correctamente.
Dimensiones después de PCA: (10, 14)
Componentes principales: 14

Primeras filas con PCA:


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14
0,-3.479708,-1.724691,0.027894,-0.496271,0.838900,-0.557671,-0.869533,-0.434740,-1.227253,0.998407,0.410380,0.432473,-0.306224,0.621639
1,0.852168,-1.158707,-0.241517,2.266068,-2.021922,-2.192913,-1.311630,0.104013,-1.443976,1.416433,-1.490762,0.633542,1.957344,1.006022
2,2.006785,-1.746857,0.452022,-0.805189,-1.042303,-1.295324,0.033497,-1.135361,-0.166058,0.931685,1.342586,-1.141820,0.741624,-0.271421
3,0.336763,-1.876300,2.674673,-0.333739,-0.997038,-0.468777,0.260501,-0.117590,-0.546064,0.357008,0.046626,-0.159693,0.026555,0.642577
4,0.496114,-1.821696,2.708222,-0.833766,-0.939412,-0.676693,0.345872,0.349223,-1.526352,-0.279776,-0.879360,-1.014374,0.225322,-0.078181


## 5. Guardar Resultados

In [ ]:
# Guardar datos preprocesados
# Sin PCA (datos escalados)
X_scaled_df.to_csv('T_new_final.csv', index=False)

# Con PCA
X_pca_df.to_csv('T_new_final_PCA.csv', index=False)

# Output de confimación de guardado
print("Archivos guardados:")
print("  - T_new_final.csv (datos escalados sin PCA)")
print("  - T_new_final_PCA.csv (datos con PCA)")

Archivos guardados:
  - T_new_final.csv (datos escalados sin PCA)
  - T_new_final_PCA.csv (datos con PCA)


## 6. Resumen

In [ ]:
# Header
print("="*80)
print("RESUMEN DEL PREPROCESAMIENTO EN PRODUCCIÓN") # Titulo
print("="*80)
print(f"\n1. DATOS DE ENTRADA:")
print(f"   - Archivo: Saratoga_desconocidos.csv")
print(f"   - Registros: {df_desconocidos.shape[0]}")
print(f"   - Features originales: {df_desconocidos.shape[1] - 1}")  # -1 por price vacío

print(f"\n2. TRANSFORMACIÓN:")
print(f"   - Features después de codificación: {df_encoded.shape[1]}")
print(f"   - Features después de escalado: {X_scaled_df.shape[1]}")
print(f"   - Features después de PCA: {X_pca_df.shape[1]}")

print(f"\n3. ARTEFACTOS UTILIZADOS:")
print(f"   - StandardScaler (preprocessor_cat.joblib)")
print(f"   - PCA con {pca_metadata['n_components']} componentes (pca_artifacts_bundle.zip)")
print(f"   - Varianza explicada: {pca_metadata['total_variance_explained']*100:.2f}%")

print(f"\n4. ARCHIVOS GENERADOS:")
print(f"   - T_new_final.csv ({X_scaled_df.shape[0]} filas x {X_scaled_df.shape[1]} columnas)")
print(f"   - T_new_final_PCA.csv ({X_pca_df.shape[0]} filas x {X_pca_df.shape[1]} columnas)")

print("\n" + "="*80)
print("PREPROCESAMIENTO EN PRODUCCIÓN COMPLETADO")
print("="*80)

RESUMEN DEL PREPROCESAMIENTO EN PRODUCCIÓN

1. DATOS DE ENTRADA:
   - Archivo: Saratoga_desconocidos.csv
   - Registros: 10
   - Features originales: 15

2. TRANSFORMACIÓN:
   - Features después de codificación: 18
   - Features después de escalado: 18
   - Features después de PCA: 14

3. ARTEFACTOS UTILIZADOS:
   - StandardScaler (preprocessor_cat.joblib)
   - PCA con 14 componentes (pca_artifacts_bundle.zip)
   - Varianza explicada: 97.11%

4. ARCHIVOS GENERADOS:
   - T_new_final.csv (10 filas x 18 columnas)
   - T_new_final_PCA.csv (10 filas x 14 columnas)

PREPROCESAMIENTO EN PRODUCCIÓN COMPLETADO


## Fin de ejecución